# Partial Derivatives Computation via Autodiff

In this notebook, we show how to use the autodiff feature of $\partial\textrm{SGP4}$. Due to the fact that it is written in `pytorch`, it automatically supports automatic differentiation via `torch.autograd`. 

In this notebook, we show how these partial derivatives can be constructed: for more advanced examples on how to use these gradients for practical applications, see the tutorials on `state_transition_matrix_computation`, `covariance_propagation`, `graident_based_optimization`, `orbit_determination`.

In [1]:
import dsgp4
import torch

We create a TLE object

In [2]:
#as always, first, we create a TLE object:
tle=[]
tle.append('0 COSMOS 2251 DEB')
tle.append('1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996')
tle.append('2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320')
tle = dsgp4.tle.TLE(tle)
print(tle)

TLE(
0 COSMOS 2251 DEB
1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996
2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320
)


Now, as shown in the `tle_propagation` tutorial, we can propagate the TLE. However, instead of using the standard API, we require `torch.autograd` to record the operations w.r.t. the time.

## Partials with respect to time

Let's compute the partials of the $\partial \textrm{SGP4}$ output w.r.t. the propagation times

### Single TLEs

Let's first see the case of single TLEs, propagated at various future times:

In [3]:
#let's take a random tensor of 10 tsince elements, where we track the gradients:
tsince=torch.rand((10,),requires_grad=True)
#the state is then:
state_teme = dsgp4.propagate(tle,
                tsinces=tsince,
                initialized=False)
#now, we can see that the gradient is tracked:
print(state_teme)

tensor([[[ 1.4324e+03, -6.9744e+03,  3.9044e+02],
         [ 1.9394e+00,  7.9228e-01,  7.1912e+00]],

        [[ 1.4096e+03, -6.9831e+03,  3.0627e+02],
         [ 1.9577e+00,  7.0238e-01,  7.1957e+00]],

        [[ 1.3535e+03, -7.0000e+03,  1.0194e+02],
         [ 2.0008e+00,  4.8392e-01,  7.2021e+00]],

        [[ 1.3436e+03, -7.0023e+03,  6.6410e+01],
         [ 2.0081e+00,  4.4589e-01,  7.2025e+00]],

        [[ 1.3465e+03, -7.0016e+03,  7.6861e+01],
         [ 2.0060e+00,  4.5708e-01,  7.2024e+00]],

        [[ 1.3664e+03, -6.9967e+03,  1.4850e+02],
         [ 1.9911e+00,  5.3373e-01,  7.2012e+00]],

        [[ 1.3373e+03, -7.0036e+03,  4.4048e+01],
         [ 2.0127e+00,  4.2196e-01,  7.2027e+00]],

        [[ 1.4265e+03, -6.9768e+03,  3.6857e+02],
         [ 1.9441e+00,  7.6892e-01,  7.1924e+00]],

        [[ 1.4082e+03, -6.9836e+03,  3.0110e+02],
         [ 1.9588e+00,  6.9686e-01,  7.1959e+00]],

        [[ 1.3772e+03, -6.9937e+03,  1.8760e+02],
         [ 1.9829e+00,  5.7555e-

Let's now retrieve the partial derivatives of the SGP4 output w.r.t. time.

Since the state is position and velocity (i.e., $[x,y,z,v_x,v_y,v_z]$), these partials will be all the elements of type:
\begin{equation}
\dfrac{d \pmb{x}}{d t}=[\dfrac{dx}{dt}, \dfrac{dy}{dt}, \dfrac{dz}{dt}, \dfrac{d^2x}{dt^2}, \dfrac{d^2y}{dt^2}, \dfrac{d^2z}{dt^2}]^T=[v_x, v_y, v_z, \dfrac{dv_x}{dt}, \dfrac{dv_y}{dt}, \dfrac{dv_z}{dt}]^T
\end{equation}


```{note}
One thing to be careful about is that $\partial\textrm{SGP4}$, mirroring the original $\textrm{SGP4}$, takes the time in minutes, and returns the state in km and km/s. Hence, the derivatives will have dimensions coherent to these, and to return to SI, conversions have to be made.
```

In [4]:
partial_derivatives = torch.zeros_like(state_teme)
for i in [0,1]:
    for j in [0,1,2]:
        tsince.grad=None
        state_teme[:,i,j].backward(torch.ones_like(tsince),retain_graph=True)
        partial_derivatives[:,i,j] = tsince.grad

#let's print to screen the partials:
print(partial_derivatives)

tensor([[[ 1.1636e+02,  4.7537e+01,  4.3147e+02],
         [-9.4609e-02,  4.6064e-01, -2.5855e-02]],

        [[ 1.1746e+02,  4.2143e+01,  4.3174e+02],
         [-9.3107e-02,  4.6124e-01, -2.0282e-02]],

        [[ 1.2005e+02,  2.9035e+01,  4.3213e+02],
         [-8.9406e-02,  4.6240e-01, -6.7516e-03]],

        [[ 1.2049e+02,  2.6754e+01,  4.3215e+02],
         [-8.8754e-02,  4.6256e-01, -4.3986e-03]],

        [[ 1.2036e+02,  2.7425e+01,  4.3215e+02],
         [-8.8946e-02,  4.6251e-01, -5.0907e-03]],

        [[ 1.1947e+02,  3.2024e+01,  4.3207e+02],
         [-9.0257e-02,  4.6217e-01, -9.8352e-03]],

        [[ 1.2077e+02,  2.5318e+01,  4.3216e+02],
         [-8.8343e-02,  4.6265e-01, -2.9176e-03]],

        [[ 1.1665e+02,  4.6136e+01,  4.3155e+02],
         [-9.4220e-02,  4.6080e-01, -2.4406e-02]],

        [[ 1.1753e+02,  4.1812e+01,  4.3176e+02],
         [-9.3015e-02,  4.6127e-01, -1.9940e-02]],

        [[ 1.1898e+02,  3.4533e+01,  4.3201e+02],
         [-9.0968e-02,  4.6196e-

### Batch TLEs

Let's now see how it works for batch TLEs. The API is basically identical:

In [5]:
#we load 6 TLEs:
inp_file="""0 PSLV DEB
1 35350U 01049QJ  22068.76869562  .00000911  00000-0  24939-3 0  9998
2 35350  98.6033  64.7516 0074531  99.8340 261.1278 14.48029442457561
0 PSLV DEB *
1 35351U 01049QK  22066.70636923  .00002156  00000-0  63479-3 0  9999
2 35351  98.8179  29.5651 0005211  45.5944 314.5671 14.44732274457505
0 SL-18 DEB
1 35354U 93014BD  22068.76520028  .00021929  00000-0  20751-2 0  9995
2 35354  75.7302 100.7819 0059525 350.7978   9.2117 14.92216400847487
0 SL-18 DEB
1 35359U 93014BJ  22068.55187275  .00025514  00000-0  24908-2 0  9992
2 35359  75.7369 156.1582 0054843  50.5279 310.0745 14.91164684775759
0 SL-18 DEB
1 35360U 93014BK  22068.44021735  .00019061  00000-0  20292-2 0  9992
2 35360  75.7343 127.2487 0071107  32.5913 327.9635 14.86997880798827
0 METEOR 2-17 DEB
1 35364U 88005Y   22067.81503681  .00001147  00000-0  84240-3 0  9995
2 35364  82.5500  92.4124 0018834 303.2489 178.0638 13.94853833332534"""
lines=inp_file.splitlines()
#let's create the TLE objects
tles=[]
for i in range(0,len(lines),3):
    data=[]
    data.append(lines[i])
    data.append(lines[i+1])
    data.append(lines[i+2])
    tles.append(dsgp4.tle.TLE(data))
#we also create 9 random times, tracking the gradients:
tsinces=torch.rand((6,),requires_grad=True)

In [6]:
#let's propagate the batch:
state_teme = dsgp4.propagate_batch(tles,
                tsinces=tsinces,
                initialized=False)

Now, let's retrieve the partial of each TLE, at each propagated time, and store them into a Nx2x3 matrix:

In [7]:
#let's retrieve the partials w.r.t. time:
partial_derivatives = torch.zeros_like(state_teme)
for i in [0,1]:
    for j in [0,1,2]:
        tsinces.grad=None
        state_teme[:,i,j].backward(torch.ones_like(tsinces),retain_graph=True)
        partial_derivatives[:,i,j] = tsinces.grad

#let's print to screen the partials:
print(partial_derivatives)

tensor([[[ 5.4025e+01, -4.2648e+01,  4.4350e+02],
         [-2.0308e-01, -4.2630e-01, -1.2250e-02]],

        [[ 2.8727e+01, -6.2794e+01,  4.4374e+02],
         [-4.1113e-01, -2.3223e-01, -5.5870e-03]],

        [[-1.0698e+02, -3.8831e+01,  4.4205e+02],
         [ 9.7992e-02, -4.8842e-01, -1.9301e-02]],

        [[-2.8367e+01, -1.0994e+02,  4.4098e+02],
         [ 4.5503e-01, -1.9636e-01, -1.7090e-02]],

        [[-8.6062e+01, -7.2340e+01,  4.4194e+02],
         [ 3.0114e-01, -3.9458e-01, -3.4948e-03]],

        [[ 4.5807e+01, -3.7642e+02, -2.2897e+02],
         [ 3.9648e-02,  2.3558e-01, -3.7947e-01]]])


## Partials with respect to TLE parameters

Let's now tackle the case in which we are interested in the partials of the $\partial\textrm{SGP4}$ output w.r.t. the TLE parameters.

### Single TLEs

We first tackle the case of single TLE, propagated at multiple times:

In this case, we want the Jacobian of the output state, w.r.t. the following TLE parameters $\textrm{TLE}=[n,e,i,\Omega,\omega,M,B^*,\dot{n},\ddot{n}]$, where:

* $n$ is the mean motion (also known as `no_kozai` in the original implementation) [rad/minute]; 
* $e$ is the eccentricity [-]; 
* $i$ is the inclination [rad]; 
* $\Omega$ is the right ascension of the ascending node [rad]; 
* $\omega$ is the argument of perigee [rad];
* $M$ is the mean anomaly [rad];
* $B^*$ is the Bstar parameter [1/earth radii]
* $\dot{n}$ mean motion first derivative [radians/$\textrm{minute}^2$]
* $\ddot{n}$ mean motion second derivative [radians/$\textrm{minute}^2$]

\begin{equation}
\dfrac{\partial \pmb{x}}{\partial \textrm{TLE}}=
\begin{bmatrix}
\frac{\partial x}{\partial B^*} & \frac{\partial x}{\partial \dot{n}} & \frac{\partial x}{\partial \ddot{n}} & \frac{\partial x}{\partial e} & \frac{\partial x}{\partial \omega} & \frac{\partial x}{\partial i} & \frac{\partial x}{\partial M} & \frac{\partial x}{\partial n} & \frac{\partial x}{\partial \Omega} \\
\\
\frac{\partial y}{\partial B^*} & \frac{\partial y}{\partial \dot{n}} & \frac{\partial y}{\partial \ddot{n}} & \frac{\partial y}{\partial e} & \frac{\partial y}{\partial \omega} & \frac{\partial y}{\partial i} & \frac{\partial y}{\partial M} & \frac{\partial y}{\partial n} & \frac{\partial y}{\partial \Omega} \\
\\
\frac{\partial z}{\partial B^*} & \frac{\partial z}{\partial \dot{n}} & \frac{\partial z}{\partial \ddot{n}} & \frac{\partial z}{\partial e} & \frac{\partial z}{\partial \omega} & \frac{\partial z}{\partial i} & \frac{\partial z}{\partial M} & \frac{\partial z}{\partial n} & \frac{\partial z}{\partial \Omega} \\
\\
\frac{\partial v_x}{\partial B^*} & \frac{\partial v_x}{\partial \dot{n}} & \frac{\partial v_x}{\partial \ddot{n}} & \frac{\partial v_x}{\partial e} & \frac{\partial v_x}{\partial \omega} & \frac{\partial v_x}{\partial i} & \frac{\partial v_x}{\partial M} & \frac{\partial v_x}{\partial n} & \frac{\partial v_x}{\partial \Omega} \\
\\
\frac{\partial v_y}{\partial B^*} & \frac{\partial v_y}{\partial \dot{n}} & \frac{\partial v_y}{\partial \ddot{n}} & \frac{\partial v_y}{\partial e} & \frac{\partial v_y}{\partial \omega} & \frac{\partial v_y}{\partial i} & \frac{\partial v_y}{\partial M} & \frac{\partial v_y}{\partial n} & \frac{\partial v_y}{\partial \Omega} \\
\\
\frac{\partial v_z}{\partial B^*} & \frac{\partial v_z}{\partial \dot{n}} & \frac{\partial v_z}{\partial \ddot{n}} & \frac{\partial v_z}{\partial e} & \frac{\partial v_z}{\partial \omega} & \frac{\partial v_z}{\partial i} & \frac{\partial v_z}{\partial M} & \frac{\partial v_z}{\partial n} & \frac{\partial v_z}{\partial \Omega} \\
\\
\frac{\partial \dot{n}}{\partial B^*} & \frac{\partial \dot{n}}{\partial \dot{n}} & \frac{\partial \dot{n}}{\partial \ddot{n}} & \frac{\partial \dot{n}}{\partial e} & \frac{\partial \dot{n}}{\partial \omega} & \frac{\partial \dot{n}}{\partial i} & \frac{\partial \dot{n}}{\partial M} & \frac{\partial \dot{n}}{\partial n} & \frac{\partial \dot{n}}{\partial \Omega} \\
\\
\frac{\partial \ddot{n}}{\partial B^*} & \frac{\partial \ddot{n}}{\partial \dot{n}} & \frac{\partial \ddot{n}}{\partial \ddot{n}} & \frac{\partial \ddot{n}}{\partial e} & \frac{\partial \ddot{n}}{\partial \omega} & \frac{\partial \ddot{n}}{\partial i} & \frac{\partial \ddot{n}}{\partial M} & \frac{\partial \ddot{n}}{\partial n} & \frac{\partial \ddot{n}}{\partial \Omega}
\end{bmatrix}
\end{equation}

In [8]:
#as always, first, we create a TLE object:
tle=[]
tle.append('0 COSMOS 2251 DEB')
tle.append('1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996')
tle.append('2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320')
tle = dsgp4.tle.TLE(tle)
print(tle)
tle_elements=dsgp4.initialize_tle(tle,with_grad=True)

TLE(
0 COSMOS 2251 DEB
1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996
2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320
)


In [9]:
#let's select 10 random times:
tsince=torch.rand((10,))
#and let's propagate:
state_teme=dsgp4.propagate(tle,tsince)

In [10]:
#now we can build the partial derivatives matrix, of shape Nx6x9 (N is the number of tsince elements, 6 is the number of elements in the state vector, and 9 is the number of elements in the TLE):
partial_derivatives = torch.zeros((len(tsince),6,9))
for k in range(len(tsince)):
    for i in range(6):
        tle_elements.grad=None
        state_teme[k].flatten()[i].backward(retain_graph=True)
        partial_derivatives[k,i,:] = tle_elements.grad
#let's print them to screen:
print(partial_derivatives)

tensor([[[-6.4523e-05,  0.0000e+00,  0.0000e+00,  9.4688e+02,  1.8970e+03,
          -1.3843e+02,  1.9118e+03, -1.3927e+04,  6.9973e+03],
         [-3.2480e-04,  0.0000e+00,  0.0000e+00,  6.4275e+03,  5.1484e+02,
          -2.9536e+01,  5.0358e+02,  7.4557e+04,  1.3641e+03],
         [-4.3288e-04,  0.0000e+00,  0.0000e+00,  7.4336e+03,  6.8559e+03,
           3.6047e+01,  6.8995e+03,  5.9987e+02,  0.0000e+00],
         [-6.4902e-07,  0.0000e+00,  0.0000e+00,  8.7493e-01, -1.4304e+00,
          -7.0725e+00, -1.4391e+00,  1.0128e+01, -5.2510e-01],
         [-3.8817e-07,  0.0000e+00,  0.0000e+00,  4.5053e+00,  7.3591e+00,
          -1.3399e+00,  7.3813e+00,  5.1095e+00,  1.9928e+00],
         [-2.4810e-06,  0.0000e+00,  0.0000e+00,  5.9443e+00, -1.3321e-01,
           2.0514e+00, -1.4851e-01,  3.8287e+01,  0.0000e+00]],

        [[-1.5784e-04,  0.0000e+00,  0.0000e+00,  9.6539e+02,  1.8644e+03,
          -2.9674e+02,  1.8790e+03, -1.3708e+04,  6.9836e+03],
         [-7.1066e-04,  0.0000e+

### Batch TLEs:

As for the time derivatives, the API stays practically identical:

In [11]:
#we load 6 TLEs:
inp_file="""0 PSLV DEB
1 35350U 01049QJ  22068.76869562  .00000911  00000-0  24939-3 0  9998
2 35350  98.6033  64.7516 0074531  99.8340 261.1278 14.48029442457561
0 PSLV DEB *
1 35351U 01049QK  22066.70636923  .00002156  00000-0  63479-3 0  9999
2 35351  98.8179  29.5651 0005211  45.5944 314.5671 14.44732274457505
0 SL-18 DEB
1 35354U 93014BD  22068.76520028  .00021929  00000-0  20751-2 0  9995
2 35354  75.7302 100.7819 0059525 350.7978   9.2117 14.92216400847487
0 SL-18 DEB
1 35359U 93014BJ  22068.55187275  .00025514  00000-0  24908-2 0  9992
2 35359  75.7369 156.1582 0054843  50.5279 310.0745 14.91164684775759
0 SL-18 DEB
1 35360U 93014BK  22068.44021735  .00019061  00000-0  20292-2 0  9992
2 35360  75.7343 127.2487 0071107  32.5913 327.9635 14.86997880798827
0 METEOR 2-17 DEB
1 35364U 88005Y   22067.81503681  .00001147  00000-0  84240-3 0  9995
2 35364  82.5500  92.4124 0018834 303.2489 178.0638 13.94853833332534"""
lines=inp_file.splitlines()
#let's create the TLE objects
tles=[]
for i in range(0,len(lines),3):
    data=[]
    data.append(lines[i])
    data.append(lines[i+1])
    data.append(lines[i+2])
    tles.append(dsgp4.tle.TLE(data))
#we also create 6 random times, tracking the gradients:
tsinces=torch.rand((6,))

In [12]:
#let's now initialize the TLEs, activating the gradient tracking for the TLE parameters:
tle_elements=dsgp4.initialize_tle(tles,with_grad=True)

In [13]:
#let's now propagate the batch of TLEs:
state_teme = dsgp4.propagate_batch(tles,tsinces)

Finally, we can build the matrix that contains the partial of the SGP4 output w.r.t. the TLE parameters, for each TLE:

In [14]:
#now we can build the partial derivatives matrix, of shape Nx6x9 (N is the number of tsince elements, 6 is the number of elements in the state vector, and 9 is the number of elements in the TLE):
partial_derivatives = torch.zeros((len(tsinces),6,9))
for k in range(len(tsinces)):
    for i in range(6):
        tle_elements[k].grad=None
        state_teme[k].flatten()[i].backward(retain_graph=True)
        partial_derivatives[k,i,:] = tle_elements[k].grad
#let's print them to screen:
print(partial_derivatives)

tensor([[[-4.7024e-04,  0.0000e+00,  0.0000e+00, -1.2286e+03,  7.9988e+02,
           3.3369e+02,  7.7575e+02, -3.1876e+04, -6.4058e+03],
         [-2.2363e-03,  0.0000e+00,  0.0000e+00,  2.3340e+03, -7.9925e+02,
          -1.5947e+02, -8.4529e+02, -6.8224e+04,  3.0828e+03],
         [ 3.4920e-03,  0.0000e+00,  0.0000e+00, -1.3916e+04,  7.0305e+03,
          -5.3544e+01,  7.0161e+03,  1.7906e+03,  0.0000e+00],
         [-1.5772e-06,  0.0000e+00,  0.0000e+00,  3.1322e+00, -3.2498e+00,
           6.6740e+00, -3.2411e+00,  1.6212e+00,  8.8900e-01],
         [-1.3577e-06,  0.0000e+00,  0.0000e+00,  6.7933e+00, -6.7337e+00,
          -3.1490e+00, -6.7346e+00, -1.0225e+01,  8.1509e-01],
         [-5.5853e-06,  0.0000e+00,  0.0000e+00, -4.2693e-01, -4.4474e-01,
          -1.1140e+00, -3.8970e-01,  3.8645e+01,  0.0000e+00]],

        [[-3.0412e-04,  0.0000e+00,  0.0000e+00, -4.9268e+03,  2.9541e+02,
           1.3106e+02,  2.9342e+02, -6.5451e+04, -3.4748e+03],
         [-3.2372e-04,  0.0000e+